In [47]:
import datetime
now = datetime.datetime.now()
print("Last executed: " + now.strftime("%Y-%m-%d %H:%M:%S"))

Last executed: 2025-02-05 11:06:41


In [48]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, validation_curve
from sklearn.metrics import root_mean_squared_error, r2_score

# preprocessors
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# pipeline
from sklearn.pipeline import Pipeline

# linear regression
from sklearn.linear_model import LinearRegression

# CART
from sklearn.tree import DecisionTreeRegressor

# random forest
from sklearn.ensemble import RandomForestRegressor

# xgboost
import xgboost
from xgboost import XGBRegressor

pd.set_option('display.max_rows', 300) # specifies number of rows to show
pd.options.display.float_format = '{:40,.4f}'.format # specifies default number format to 4 decimal places
plt.style.use('ggplot') # specifies that graphs should use ggplot styling
%matplotlib inline

In [31]:
# check the library version before we start
print("xgboost version:{}".format(xgboost.__version__))
print("sklearn version:{}".format(sklearn.__version__))

xgboost version:2.1.1
sklearn version:1.5.2


In [32]:
bike_rental = pd.read_csv('https://raw.githubusercontent.com/huanfachen/Spatial_Data_Science/main/Dataset/daily_count_bike_rental.csv')
# drop the year variable as it is not useful
bike_rental = bike_rental.drop(['yr'], axis=1)

In [33]:
bike_rental.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   cnt              731 non-null    int64  
 1   season           731 non-null    object 
 2   mnth             731 non-null    object 
 3   holiday          731 non-null    object 
 4   weekday          731 non-null    object 
 5   workingday       731 non-null    object 
 6   weathersit       731 non-null    object 
 7   temp             731 non-null    float64
 8   hum              731 non-null    float64
 9   windspeed        731 non-null    float64
 10  days_since_2011  731 non-null    int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 62.9+ KB


In [34]:
# print a few rows of this dataset
bike_rental.head()

,cnt,season,mnth,holiday,weekday,workingday,weathersit,temp,hum,windspeed,days_since_2011
0,985,SPRING,JAN,NO HOLIDAY,SAT,NO WORKING DAY,MISTY,8.1758,80.5833,10.7499,0
1,801,SPRING,JAN,NO HOLIDAY,SUN,NO WORKING DAY,MISTY,9.0835,69.6087,16.6521,1
2,1349,SPRING,JAN,NO HOLIDAY,MON,WORKING DAY,GOOD,1.2291,43.7273,16.6367,2
3,1562,SPRING,JAN,NO HOLIDAY,TUE,WORKING DAY,GOOD,1.4000,59.0435,10.7398,3
4,1600,SPRING,JAN,NO HOLIDAY,WED,WORKING DAY,GOOD,2.6670,43.6957,12.5223,4


In [35]:
bike_rental.isnull().sum()

cnt                0
season             0
mnth               0
holiday            0
weekday            0
workingday         0
weathersit         0
temp               0
hum                0
windspeed          0
days_since_2011    0
dtype: int64

In [36]:
random_state_split = 100
train_x, test_x, train_y, test_y = train_test_split(bike_rental.drop(['cnt'], axis = 1), bike_rental.cnt, random_state=random_state_split)

In [37]:
# The missing values of a numeric feature will be replaced by the 'mean' value of the feature.
# The missing values of a categorical feature will be replaced by the a 'constant' value. If 'constant' value is not specified, it is default at '0' for numerical data or 'np.nan' for categorical data.

numeric_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='mean'))
      ,('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='constant'))
      ,('encoder', OneHotEncoder(drop='first'))
])

In [38]:
numeric_features = ['temp', 'hum', 'windspeed', 'days_since_2011']
categorical_features = ['season', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit']
preprocessor = ColumnTransformer(
   transformers=[
    ('numeric', numeric_transformer, numeric_features)
   ,('categorical', categorical_transformer, categorical_features)
])

In [39]:
pipeline = Pipeline(steps = [
   ('preprocessor', preprocessor),
   ('regressor',DecisionTreeRegressor())
])

In [40]:
cart_model = pipeline.fit(train_x, train_y)
# this will visualise the pipeline
print(cart_model)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['temp', 'hum', 'windspeed',
                                                   'days_since_2011']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first'))]),
   

In [41]:
cart_model = pipeline.fit(train_x, train_y)
# this will visualise the pipeline
print(cart_model)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['temp', 'hum', 'windspeed',
                                                   'days_since_2011']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first'))]),
   

In [52]:
print("RMSE on the training data:")
print(root_mean_squared_error(train_y, cart_model.predict(train_x)))
print("RMSE on the testing data:")
print(root_mean_squared_error(test_y, cart_model.predict(test_x)))

RMSE on the training data:
0.0
RMSE on the testing data:
890.9913800617054


In [19]:
print("R2 on the training data:")
print(r2_score(train_y, cart_model.predict(train_x)))
print("R2 on the testing data:")
print(r2_score(test_y, cart_model.predict(test_x)))

R2 on the training data:
1.0
R2 on the testing data:
0.7190635421535585


In [20]:
# we fix the random_state in DecisionTreeRegressor() so that the result of GridSearchCV is the same in different runs
cart_pipeline = Pipeline(steps = [
  ('preprocessor', preprocessor),
  ('regressor', DecisionTreeRegressor(random_state=123))
])

cart_pipeline.fit(train_x, train_y)

# grid_params is the range of each hyperparameter
grid_params = {
  'regressor__max_depth': [10,20,30,40,50], 
  'regressor__min_samples_split': [2,4,6,8,10]
}
search = GridSearchCV(cart_pipeline, grid_params)
search.fit(train_x, train_y)
print("Best R2 Score: ", search.best_score_)
print("Best Params: ", search.best_params_)

Best R2 Score:  0.7578489598345406
Best Params:  {'regressor__max_depth': 10, 'regressor__min_samples_split': 8}


In [53]:
regressors = {
    'Linear': LinearRegression(),
    'CART': DecisionTreeRegressor(),
    'RF': RandomForestRegressor(),
    'XGB': XGBRegressor()
}

# a dict to store the R2 of training and testing data
dict_results = dict()

for name, regressor in regressors.items():
    pipeline = Pipeline(steps = [
               ('preprocessor', preprocessor)
              ,('regressor', regressor)
           ])
    model = pipeline.fit(train_x, train_y)
    predictions = model.predict(test_x)
    dict_results[name] = [model.score(train_x, train_y), model.score(test_x, test_y), model.score(train_x, train_y) - model.score(test_x, test_y)]

# transform dict_models to dataframe
df_models = pd.DataFrame.from_dict(dict_results, orient='index', columns=['R2_train_data', 'R2_test_data', 'R2_diff'])
df_models

,R2_train_data,R2_test_data,R2_diff
Linear,0.8576,0.7992,0.0584
CART,1.0000,0.7364,0.2636
RF,0.9830,0.8834,0.0996
XGB,1.0000,0.8763,0.1236


In [22]:
!pip install mlxtend==0.21.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.6 MB/s eta 0:00:00a 0:00:01


In [23]:
from mlxtend.evaluate import bias_variance_decomp

In [24]:
# one-hot encoding
bike_rentail_numeric = pd.get_dummies(bike_rental)
bike_rental_final = bike_rentail_numeric.drop(['season_SPRING', 'mnth_JAN', 'holiday_NO HOLIDAY', 'weekday_MON', 'workingday_WORKING DAY', 'weathersit_GOOD'], axis=1)

# train-test split
random_state_split = 100
train_x, test_x, train_y, test_y = train_test_split(bike_rental_final.drop(['cnt'], axis = 1), bike_rental_final.cnt, random_state=random_state_split)

In [25]:
# create and train a DecisionTreeRegressor model using numpy array datasets. compute the total error, bias, and variance of this model
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        DecisionTreeRegressor(random_state=0), train_x.to_numpy(), train_y.to_numpy(), test_x.to_numpy(), test_y.to_numpy(), 
        loss='mse',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)

Average expected loss: 915617.759
Average bias: 406469.732
Average variance: 509148.027


In [26]:
random_seed = 1233
regressors = {
    'Linear': LinearRegression(),
    'CART': DecisionTreeRegressor(random_state = random_seed),
    'RF': RandomForestRegressor(random_state = random_seed),
    'XGB': XGBRegressor(random_state = random_seed)
}

# a dict to store the R2 of training and testing data
dict_results = dict()

for name, regressor in regressors.items():
    avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        regressor, train_x.to_numpy(), train_y.to_numpy(), test_x.to_numpy(), test_y.to_numpy(), 
        loss='mse',
        random_seed=123,
        num_rounds=50)
    dict_results[name] = [avg_expected_loss, avg_bias, avg_var]

# transform dict_models to dataframe
df_models = pd.DataFrame.from_dict(dict_results, orient='index', columns=['Total loss', 'Bias (or Bias^2)', 'Variance'])
df_models

,Total loss,Bias (or Bias^2),Variance
Linear,"739,779.1697","705,745.3435","34,033.8262"
CART,"936,825.7866","433,586.5275","503,239.2591"
RF,"488,162.1634","408,270.6061","79,891.5573"
XGB,"526,975.2811","388,590.2300","138,385.0511"
